# Finetune ruGPT3Small on your data


Tutorial and explications are here: https://habr.com/ru/company/sberbank/blog/528966/

First, connect Google Drive with the Colab notebook to access your data directly from GDrive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## Add data to colab


In [ ]:
# this is a dataset that was provided by Sberbank
# !wget https://raw.githubusercontent.com/FurkanArslan/twitter-text-classification/master/twitter_sentiment_corpus.csv
# data_path = "twitter_sentiment_corpus.csv"

In [ ]:
# here your data goes 
# previously I removed all usernames starting with @ and loaded my Twitter data to my GDrive disk
# The format is a simple CSV/TSV table

data_path = "/content/gdrive/My Drive/MyTwitter/my_twitter_11_2020.tsv"

## Prepare data

In [ ]:
import pandas as pd

data = pd.read_csv(data_path, sep='\t')

# replace nans with a blank space
data = data.fillna(' ') 

print('Total tweets: ', len(data))
data.head()

In [ ]:
# remove all empty tweets
data = data[data['tweet_clean'] != ' ']
print('Tweets after filtering: ', len(data))

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
data.shape

In [ ]:
# select 1000 random items for validation set
val_ind = random.sample(range(data.shape[0]), 1000)

In [ ]:
train = [data.iloc[i]['tweet_clean'] for i in range(len(data)) if i not in val_ind][:10000]
valid = [data.iloc[i]['tweet_clean'] for i in range(len(data)) if i in val_ind]

In [ ]:
len(train), len(valid)

In [ ]:
# save the datasets to the disk
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung 15 minutes to train the model on 10 epochs. 

In [ ]:
!python pretrain_transformers.py \
    --output_dir=comment_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 10 \
    --block_size 2048 \
    --overwrite_output_dir

## Check our model

Here I added a script from this Notebook made by unknown Habr commentator: 
https://colab.research.google.com/drive/1r5ufZF9SZPowAs0K8pQzESIjbcb1WTMd

It's a curious stuff as it was made to generate horoscopes.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("comment_model") # here goes the folder name where our new model is saved
model = GPT2LMHeadModel.from_pretrained("comment_model")
model.to("cuda")

In [ ]:
import copy

# to make the generated texts better, we remove "bad" tokens from the model, some junk symbols such as html tags, links etc. 

bad_word_ids = [
    [203], # \n
    [225], # weird space 1
    [28664], # weird space 2
    [13298], # weird space 3
    [206], # \r
    [49120], # html
    [25872], # http
    [3886], # amp
    [38512], # nbsp
    [10], # &
    [5436], # & (another)
    [5861], # http
    [372], # yet another line break
    [421, 4395], # МСК
    [64], # \
    [33077], # https
    [1572], # ru
    [11101], # Источник
]

def gen_fragment(context, bad_word_ids=bad_word_ids, print_debug_output=False, temperature=1.0, max_length=75, min_length=50):
    input_ids = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt").to("cuda")
    input_ids = input_ids[:, -1700:]
    input_size = input_ids.size(1)
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length + input_size,
        min_length=min_length + input_size,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        temperature=1.0,
        pad_token_id=0,
        eos_token_id=2,
        bad_words_ids=bad_word_ids,
        no_repeat_ngram_size=6
    )
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    generated_sequence = output_sequences[0].tolist()[input_size:]
    if print_debug_output:
        for idx in generated_sequence:
            print(idx, tokenizer.decode([idx], clean_up_tokenization_spaces=True).strip())
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    text = text[: text.find("</s>")]
    text = text[: text.rfind(".") + 1]
    return context + text

In [ ]:
beginning = "Я люблю такие"
print(gen_fragment(beginning, temperature=1.0, max_length=40))

This is the original Sberbank's funcion to generate texts. I think it's output has too many artefacts but still good. 


In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --k=0 \
    --p=1.0 \
    --length=40

## Some examples I got from the gen_fragment function



Context >>> В Мадриде есть

ruGPT:
В Мадриде есть  целый квартал, где сосредоточено все самое элитное. В старом городе много церквей и соборов. Но когда приезжаешь в Сан-Себастьян и видишь Мадрид как бы со стороны, то кажется, что ты на вершине горы.


Context >>> Вчера вечером я

ruGPT:
Вчера вечером я  видела, как два чувака лет 30 сидели тихо в кафе в парке, пили пиво и иногда ругались матом. В общем, весь вечер гуляли, никого не трогали.


Context >>> Моя любимая книга

ruGPT:
Моя любимая книга о жизни :) Когда читаешь ее с таким восторженным взглядом, даже не знаешь, что там на самом деле. Книга очень клевая. Только герои не очень. Особенно Вуди Аллен какой-то скучный.


Context >>> Мне нравится

ruGPT: Мне нравится этот актер. Почему? Он хороший, умный и добрый.


## Some examples from Sberbank function

Context >>> В Мадриде есть

ruGPT В Мадриде есть так называемые Tower Rules Officers - это массовые рейды вдоль побережья, каждые выходные. Но помимо всего, толпы народа, адово пыль.


Context >>> Вчера вечером я 

ruGPT: Вчера вечером я гуляла по лесу и наткнулась на мужика лет 40-50 с косичками. Фотографироваться не стала, пошла дальше.


Context >>> Моя любимая книга 

ruGPT: Моя любимая книга у моего учителя русского. Ребенок провел за написание одной страницы в месяц 8 часов, 4 страницы над каждой. Итог — 6 книг из 6,5.

Context >>> Мне нравится

ruGPT: Мне нравится Тимур Родригес. И песня запоминается надолго.